In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imageio
import os
import random
from utils import *

In [ ]:
images_folder_path = './Images/1'
images = load_images(images_folder_path)


kps, dess, pw_correspondences = pairwise_correspondences(images, ratio=0.6)
pw_homographies, pw_inliers, pw_outliers, matches, best_matches = pairwise_ransac(pw_correspondences)
params = bundle_adjust(pw_inliers, matches)